In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
tf.config.list_physical_devices('GPU')

[]

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
import tensorflow as tf
#Dense
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        self.units = units
        super(MyDense, self).__init__(**kwargs)

    #build方法一般定义Layer需要被训练的参数。    
    def build(self, input_shape): 
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='w')
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='b')
        super(MyDense,self).build(input_shape) # 相当于设置self.built = True

    #call方法一般定义正向传播运算逻辑，__call__方法调用了它。    
    def call(self, inputs): 
        return tf.matmul(inputs, self.w) + self.b

    #如果要让自定义的Layer通过Functional API 组合成模型时可以序列化，需要自定义get_config方法。
    def get_config(self):  
        config = super(MyDense, self).get_config()
        config.update({'units': self.units})
        return config

In [5]:
from sklearn import datasets
iris = datasets.load_iris()
data = iris.data
labels = iris.target
# from sklearn.preprocessing import MinMaxScaler
# data=MinMaxScaler().fit_transform(data)

In [6]:
data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [7]:
labels#(150,3)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
#网络   函数式构建的网络
inputs = tf.keras.Input(shape=(4,))  
x = MyDense(units=16)(inputs) 
x = tf.nn.tanh(x) 
x = MyDense(units=3)(x) #0,1,2
# x= tf.keras.layers.Dense(16)(x)
predictions = tf.nn.softmax(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)


In [9]:
#shuffle:

In [10]:
import numpy as np

In [11]:
data = np.concatenate((data,labels.reshape(150,1)),axis=-1)
np.random.shuffle(data)

In [12]:
labels = data[:,-1]
data = data[:,:4]

In [13]:
#labels  ==[1,0,0]

In [14]:
#优化器 Adam
#损失函数 交叉熵损失函数
#评估函数 #acc


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

#keras
model.fit(data, labels, batch_size=32, epochs=100,shuffle=True)

Epoch 1/100
5/5 [==============================] - 0s 3ms/step - loss: 1.1004 - sparse_categorical_accuracy: 0.3333
Epoch 2/100
5/5 [==============================] - 0s 797us/step - loss: 1.0976 - sparse_categorical_accuracy: 0.3333
Epoch 3/100
5/5 [==============================] - 0s 798us/step - loss: 1.0950 - sparse_categorical_accuracy: 0.3333
Epoch 4/100
5/5 [==============================] - 0s 997us/step - loss: 1.0925 - sparse_categorical_accuracy: 0.3333
Epoch 5/100
5/5 [==============================] - 0s 598us/step - loss: 1.0900 - sparse_categorical_accuracy: 0.3333
Epoch 6/100
5/5 [==============================] - 0s 997us/step - loss: 1.0871 - sparse_categorical_accuracy: 0.3200
Epoch 7/100
5/5 [==============================] - 0s 598us/step - loss: 1.0843 - sparse_categorical_accuracy: 0.3467
Epoch 8/100
5/5 [==============================] - 0s 997us/step - loss: 1.0810 - sparse_categorical_accuracy: 0.3533
Epoch 9/100
5/5 [==============================] - 0s 798u

In [15]:
# _custom_objects = {
#     "Mylayer" :  Line,
# }

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
my_dense (MyDense)           (None, 16)                80        
_________________________________________________________________
tf_op_layer_Tanh (TensorFlow [(None, 16)]              0         
_________________________________________________________________
my_dense_1 (MyDense)         (None, 3)                 51        
_________________________________________________________________
tf_op_layer_Softmax (TensorF [(None, 3)]               0         
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.save('keras_model_tf_version.h5')

In [20]:
_custom_objects = {
    "MyDense" :  MyDense,
    
}

In [21]:
new_model = tf.keras.models.load_model("keras_model_tf_version.h5",custom_objects=_custom_objects)

In [22]:
y_pred = new_model.predict(data)

In [23]:
y_pred = new_model.predict(data)

In [24]:
np.argmax(y_pred,axis=1)

array([1, 2, 0, 2, 2, 2, 1, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 2, 0, 1, 0, 2,
       0, 2, 0, 0, 1, 1, 2, 0, 2, 1, 1, 2, 2, 2, 1, 1, 0, 2, 2, 0, 0, 1,
       0, 2, 1, 2, 0, 2, 1, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 0, 2, 2, 0, 0,
       0, 2, 2, 1, 1, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2,
       2, 0, 1, 1, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 0, 2, 1, 2, 0, 0, 0,
       1, 2, 0, 0, 0, 2, 1, 0, 2, 2, 1, 2, 2, 1, 1, 0, 1, 0, 1, 2, 1, 1,
       1, 1, 0, 0, 2, 1, 1, 2, 1, 2, 1, 2, 2, 0, 2, 0, 1, 1], dtype=int64)

In [25]:
labels

array([1., 2., 0., 2., 2., 2., 1., 0., 0., 1., 1., 1., 2., 1., 1., 1., 2.,
       2., 0., 1., 0., 2., 0., 2., 0., 0., 1., 1., 2., 0., 2., 1., 1., 2.,
       2., 2., 1., 1., 0., 2., 2., 0., 0., 1., 0., 2., 1., 2., 0., 1., 1.,
       1., 2., 1., 0., 1., 0., 1., 2., 1., 2., 0., 2., 2., 0., 0., 0., 2.,
       2., 1., 1., 2., 0., 0., 2., 0., 0., 0., 0., 2., 0., 2., 2., 0., 1.,
       2., 2., 2., 2., 0., 1., 1., 1., 0., 0., 2., 0., 0., 2., 0., 0., 0.,
       1., 0., 2., 1., 2., 0., 0., 0., 1., 2., 0., 0., 0., 2., 1., 0., 1.,
       2., 1., 2., 2., 1., 1., 0., 1., 0., 1., 2., 1., 1., 1., 1., 0., 0.,
       2., 1., 1., 2., 1., 2., 1., 2., 2., 0., 2., 0., 1., 1.])

In [26]:
y_pred

array([[2.00078618e-02, 7.66487658e-01, 2.13504568e-01],
       [6.27030095e-04, 1.04543991e-01, 8.94828975e-01],
       [9.66335952e-01, 3.35840806e-02, 7.99577901e-05],
       [2.20586895e-03, 2.40967438e-01, 7.56826699e-01],
       [2.29930854e-04, 4.56604995e-02, 9.54109609e-01],
       [2.30546528e-03, 2.40581542e-01, 7.57112980e-01],
       [2.40447652e-02, 7.49658763e-01, 2.26296425e-01],
       [9.59850848e-01, 4.00426909e-02, 1.06369684e-04],
       [9.67726350e-01, 3.22008431e-02, 7.28223094e-05],
       [2.45897323e-02, 7.90507913e-01, 1.84902430e-01],
       [6.45587891e-02, 8.42842579e-01, 9.25986767e-02],
       [2.77159531e-02, 7.31590986e-01, 2.40693033e-01],
       [2.80742184e-03, 2.73657978e-01, 7.23534584e-01],
       [3.95808108e-02, 8.02907705e-01, 1.57511488e-01],
       [2.10131668e-02, 6.86011016e-01, 2.92975783e-01],
       [4.24388088e-02, 8.21250319e-01, 1.36310950e-01],
       [5.65340044e-04, 9.05739442e-02, 9.08860683e-01],
       [3.37491231e-03, 3.62260